In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

In [2]:
spark = (
    SparkSession.builder
        .master("local")
        .appName("Practice Kaggle Data Read")
        .config("spark.jars.packages", "org.postgresql:postgresql:42.7.3")
        .getOrCreate()
)

:: loading settings :: url = jar:file:/Users/jasminewilliams/Library/Python/3.9/lib/python/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/jasminewilliams/.ivy2/cache
The jars for the packages stored in: /Users/jasminewilliams/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-12ccb19f-f71b-458a-a61f-7232c928c360;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.3 in central
	found org.checkerframework#checker-qual;3.42.0 in central
:: resolution report :: resolve 339ms :: artifacts dl 22ms
	:: modules in use:
	org.checkerframework#checker-qual;3.42.0 from central in [default]
	org.postgresql#postgresql;42.7.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------

In [6]:
#configs
json_location = "/Users/jasminewilliams/git/chatgpt_exam/chatgpt_pipeline/data/unzip/900k Definitive Spotify Dataset.json"
df = spark.read.json(json_location)
df.show()

+------------+--------------------+--------------+------------+------+--------+-------+----------------+-----------------+------------------------+--------------+------------------------------+----------------+--------------------------+-------------------+------------------------+----------------+------+------+--------+-------------+----------+------------+------------+--------------------+-----------+------------+--------------+--------+--------------------+--------------------+
|Acousticness|               Album|     Artist(s)|Danceability|Energy|Explicit|  Genre|Good for Driving|Good for Exercise|Good for Morning Routine|Good for Party|Good for Relaxation/Meditation|Good for Running|Good for Social Gatherings|Good for Work/Study|Good for Yoga/Stretching|Instrumentalness|   Key|Length|Liveness|Loudness (db)|Popularity|Positiveness|Release Date|       Similar Songs|Speechiness|       Tempo|Time signature| emotion|                song|                text|
+------------+--------------

In [7]:
df.filter("song ilike 'Runaway Girl'").show()

+------------+----------------+-------------------+------------+------+--------+---------------+----------------+-----------------+------------------------+--------------+------------------------------+----------------+--------------------------+-------------------+------------------------+----------------+------+------+--------+-------------+----------+------------+------------+--------------------+-----------+------------+--------------+--------+------------+--------------------+
|Acousticness|           Album|          Artist(s)|Danceability|Energy|Explicit|          Genre|Good for Driving|Good for Exercise|Good for Morning Routine|Good for Party|Good for Relaxation/Meditation|Good for Running|Good for Social Gatherings|Good for Work/Study|Good for Yoga/Stretching|Instrumentalness|   Key|Length|Liveness|Loudness (db)|Popularity|Positiveness|Release Date|       Similar Songs|Speechiness|       Tempo|Time signature| emotion|        song|                text|
+------------+------------

In [9]:
#format columns
format_v1_cols = [c.lower().replace(" ","_") for c in df.columns]
formatted_cols = [c.replace('(','').replace(')','').replace('/',"_") for c in format_v1_cols]
df = df.toDF(*formatted_cols)
df.show()

+------------+--------------------+--------------+------------+------+--------+-------+----------------+-----------------+------------------------+--------------+------------------------------+----------------+--------------------------+-------------------+------------------------+----------------+------+------+--------+------------+----------+------------+------------+--------------------+-----------+------------+--------------+--------+--------------------+--------------------+
|acousticness|               album|       artists|danceability|energy|explicit|  genre|good_for_driving|good_for_exercise|good_for_morning_routine|good_for_party|good_for_relaxation_meditation|good_for_running|good_for_social_gatherings|good_for_work_study|good_for_yoga_stretching|instrumentalness|   key|length|liveness| loudness_db|popularity|positiveness|release_date|       similar_songs|speechiness|       tempo|time_signature| emotion|                song|                text|
+------------+----------------

In [5]:
df.filter("song ilike 'Runaway Girl'").show()

+------------+----------------+-------------------+------------+------+--------+---------------+----------------+-----------------+------------------------+--------------+------------------------------+----------------+--------------------------+-------------------+------------------------+----------------+------+------+--------+-------------+----------+------------+------------+--------------------+-----------+------------+--------------+--------+------------+--------------------+
|Acousticness|           Album|          Artist(s)|Danceability|Energy|Explicit|          Genre|Good for Driving|Good for Exercise|Good for Morning Routine|Good for Party|Good for Relaxation/Meditation|Good for Running|Good for Social Gatherings|Good for Work/Study|Good for Yoga/Stretching|Instrumentalness|   Key|Length|Liveness|Loudness (db)|Popularity|Positiveness|Release Date|       Similar Songs|Speechiness|       Tempo|Time signature| emotion|        song|                text|
+------------+------------

In [10]:
#data type conversion
misconfigured_types = {
    "release_date": "timestamp",
    "popularity": "int",
    "positiveness": "int",
    "energy": "int",
    "danceability": "int",
    "speechiness": "int",
    "liveness": "int",
    "acousticness": "int",
    "instrumentalness": "int"
}

for column_name, new_type in misconfigured_types.items():
    df = df.withColumn(column_name, f.col(column_name).cast(new_type))


In [11]:
#assumption 
df = df.filter('release_date is not null')

In [11]:
df = df.drop_duplicates()

### Data profiling

In [ ]:
df.count()

498052

In [ ]:
#Release date holding most of the missing values
col_missing_count_df = df.select([f.count(f.when(f.isnull(c), c)).alias(c) for c in df.columns])
col_missing_count_df.show()
#30% of release date values are null
#14 cols of the pk are null song

+------------+-----+-------+------------+------+--------+-----+----------------+-----------------+------------------------+--------------+------------------------------+----------------+--------------------------+-------------------+------------------------+----------------+---+------+--------+-----------+----------+------------+------------+-------------+-----------+-----+--------------+-------+----+----+
|acousticness|album|artists|danceability|energy|explicit|genre|good_for_driving|good_for_exercise|good_for_morning_routine|good_for_party|good_for_relaxation_meditation|good_for_running|good_for_social_gatherings|good_for_work_study|good_for_yoga_stretching|instrumentalness|key|length|liveness|loudness_db|popularity|positiveness|release_date|similar_songs|speechiness|tempo|time_signature|emotion|song|text|
+------------+-----+-------+------------+------+--------+-----+----------------+-----------------+------------------------+--------------+------------------------------+-----------

In [ ]:
df.select(['artists','song']).distinct().count() #498052 - pk matches count

498052

In [ ]:
df.select(["artists", "song", "genre", "release_date"]).show()

+--------------+--------------------+-------+-------------------+
|       artists|                song|  genre|       release_date|
+--------------+--------------------+-------+-------------------+
|           !!!|Even When the Wat...|hip hop|2013-04-29 00:00:00|
|           !!!|  One Girl / One Boy|hip hop|2013-04-29 00:00:00|
|           !!!|   Pardon My Freedom|hip hop|2004-06-08 00:00:00|
|           !!!|                 Ooo|hip hop|2015-10-16 00:00:00|
|           !!!|          Freedom 15|hip hop|2015-10-16 00:00:00|
|           !!!|       All U Writers|hip hop|2015-04-27 00:00:00|
|           !!!|        Serbia Drums|hip hop|2019-08-30 00:00:00|
|           !!!|    Must Be the Moon|hip hop|2007-03-05 00:00:00|
|           !!!|                Slyd|hip hop|2013-04-29 00:00:00|
|           !!!|Hello? Is This Th...|hip hop|2004-06-08 00:00:00|
|           !!!|          Myth Takes|hip hop|2007-03-05 00:00:00|
|           !!!|     Heart of Hearts|hip hop|2007-03-05 00:00:00|
|         

In [ ]:
#Cannot use more data not eough space in memory 
#
from ydata_profiling import ProfileReport    
a = ProfileReport(df.select(["artists", "song", "genre"]), minimal = True)
a.to_file("test.html")

25/06/02 17:09:03 WARN CacheManager: Asked to cache already cached data.        
25/06/02 17:09:31 WARN CacheManager: Asked to cache already cached data.        le: artists]
25/06/02 17:09:57 WARN CacheManager: Asked to cache already cached data.        le: song]   
Export report to file: 100%|██████████| 1/1 [00:00<00:00, 653.11it/s]


### Write

In [ ]:
a.to_file("test.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]25/06/02 17:08:19 WARN CacheManager: Asked to cache already cached data.
25/06/02 17:08:20 WARN CacheManager: Asked to cache already cached data.
ERROR:root:KeyboardInterrupt while sending command.                (3 + 1) / 11]
Traceback (most recent call last):
  File "/Users/jasminewilliams/Library/Python/3.9/lib/python/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/Users/jasminewilliams/Library/Python/3.9/lib/python/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt
Summarize dataset:   0%|          | 0/9 [00:13<?, ?it/s]


KeyboardInterrupt: 

In [21]:
partition_cols = ["artists", "song", "genre", "release_date"]
df = df.repartition(*partition_cols)
df.show()

+------------+--------------------+--------------------+------------+------+--------+--------------------+----------------+-----------------+------------------------+--------------+------------------------------+----------------+--------------------------+-------------------+------------------------+----------------+------+------+--------+------------+----------+------------+-------------------+--------------------+-----------+------------+--------------+-------+--------------------+--------------------+
|acousticness|               album|             artists|danceability|energy|explicit|               genre|good_for_driving|good_for_exercise|good_for_morning_routine|good_for_party|good_for_relaxation_meditation|good_for_running|good_for_social_gatherings|good_for_work_study|good_for_yoga_stretching|instrumentalness|   key|length|liveness| loudness_db|popularity|positiveness|       release_date|       similar_songs|speechiness|       tempo|time_signature|emotion|                song|  

In [22]:
df.write.parquet('chatgpt_pipeline/data/parquet/')

In [4]:
!java -version

openjdk version "1.8.0_452"
OpenJDK Runtime Environment (Temurin)(build 1.8.0_452-b09)
OpenJDK 64-Bit Server VM (Temurin)(build 25.452-b09, mixed mode)


In [5]:
!pyspark --version

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.5.5
      /_/
                        
Using Scala version 2.12.18, OpenJDK 64-Bit Server VM, 1.8.0_452
Branch HEAD
Compiled by user ubuntu on 2025-02-23T20:30:46Z
Revision 7c29c664cdc9321205a98a14858aaf8daaa19db2
Url https://github.com/apache/spark
Type --help for more information.


In [3]:
odbc_df = spark.read.parquet("../data/parquet")
odbc_df.show()

25/06/03 16:33:45 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------+--------------------+--------------------+------------+------+--------+--------------------+----------------+-----------------+------------------------+--------------+------------------------------+----------------+--------------------------+-------------------+------------------------+----------------+------+------+--------+------------+----------+------------+-------------------+--------------------+-----------+------------+--------------+-------+--------------------+--------------------+
|acousticness|               album|             artists|danceability|energy|explicit|               genre|good_for_driving|good_for_exercise|good_for_morning_routine|good_for_party|good_for_relaxation_meditation|good_for_running|good_for_social_gatherings|good_for_work_study|good_for_yoga_stretching|instrumentalness|   key|length|liveness| loudness_db|popularity|positiveness|       release_date|       similar_songs|speechiness|       tempo|time_signature|emotion|                song|  

In [9]:
odbc_df = odbc_df.drop("similar_songs")

In [10]:
odbc_df.write \
.format("jdbc") \
.option("url", "jdbc:postgresql://localhost:5432/testdb") \
.option("dbtable", "songs") \
.option("user", "jw9582") \
.option("password", "pass") \
.option("driver", "org.postgresql.Driver") \
.mode("overwrite") \
.save()

In [ ]:
Py4JJavaError: An error occurred while calling o83.save.
: java.lang.ClassNotFoundException: org.postgresql.Driver